In [1]:
!CUDA_LAUNCH_BLOCKING=1
!TORCH_USE_CUDA_DSA=1

In [2]:
import os

# 设置环境变量，仅使用编号为 1 的 GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import torch
device = torch.cuda.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 设置默认的CUDA设备
torch.cuda.set_device(device)

/home/yuj49/anaconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device

In [4]:
train_names = [
    "../data/GPT3.5/msmarco_train_100.jsonl",
    "../data/GPT3.5/msmarco_train_200.jsonl",
    "../data/GPT3.5/msmarco_train_300.jsonl",
    "../data/GPT3.5/msmarco_train_400.jsonl",
    "../data/GPT3.5/msmarco_train_500.jsonl",
    "../data/GPT3.5/msmarco_train_600.jsonl",
    "../data/GPT3.5/msmarco_train_700.jsonl",
    "../data/GPT3.5/msmarco_train_800.jsonl",
    "../data/GPT3.5/msmarco_train_900.jsonl",
    "../data/GPT3.5/msmarco_train_1000.jsonl",
    "../data/GPT3.5/msmarco_train_1100.jsonl"
]

val_name = "../data/GPT3.5/msmarco_test.jsonl"

True

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch
# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the model
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
model.to(device)
model.device

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


device(type='cuda', index=0)

In [ ]:
# Prepare the dataset
def preprocess_function(examples):
   
    inputs = ["query: " + q + " documents: " + " ".join(d) for q, d in zip(examples['query'], examples['unsorted_docs'])]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    # # 打印出一些处理后的输入长度
    # print("处理后的输入长度：", [len(i) for i in inputs])

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['reason'], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

data_files = {
    "train": train_names,
    "validation": val_name
}
dataset = load_dataset("json", data_files=data_files)
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# remove device
class CustomTrainer(Trainer):
    def _move_model_to_device(self, model, device):
        # 这个函数将被调用以尝试移动模型到设备，我们在这里不执行任何操作
        # 保留这个空函数即可阻止原始的.to(device)行为
        pass
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./t5_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    # predict_with_generate=True
)


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

# Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     tokenizer=tokenizer
# )

# Train the model
trainer.train()


In [3]:
from rouge_score import rouge_scorer
from datasets import load_dataset
from alignscore import AlignScore
# Function to compute ROUGE scores
def compute_rouge_scores(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]

    # Aggregate scores
    aggregated_scores = {}
    for score in scores:
        for key in score:
            if key in aggregated_scores:
                aggregated_scores[key].append(score[key].fmeasure)
            else:
                aggregated_scores[key] = [score[key].fmeasure]

    # Calculate average scores
    for key in aggregated_scores:
        aggregated_scores[key] = sum(aggregated_scores[key]) / len(aggregated_scores[key])

    return aggregated_scores
def compute_align_scores(predictions, references):
    scorer = AlignScore(model='roberta-base', batch_size=32, device='cuda:0', ckpt_path='/home/yuj49/BioLaySumm2024-evaluation_scripts/models/AlignScore/AlignScore-base.ckpt', evaluation_mode='nli_sp')
    score = scorer.score(contexts=references,claims=predictions)
    return sum(score)/len(score)

# Example usage
predictions = ["This document is about climate change because it discusses environmental impacts."]
references = ["The reason this document is ranked high is its detailed discussion on climate change impacts."]
rouge_scores = compute_rouge_scores(predictions, references)
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge1': 0.5384615384615385, 'rouge2': 0.25, 'rougeL': 0.4615384615384615}


In [11]:

def generate_reason(query, documents):
    # Prepare the input text
    input_text = f"query: {query} documents: " + " ".join(documents)
    
    # Encode the inputs
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    input_ids = inputs["input_ids"].to(device) 
    # print(inputs.device)
    # Generate output using the model
    outputs = model.generate(input_ids, max_length=150, num_return_sequences=1)
    
    # Decode and print the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [15]:
# Assuming 'tokenized_datasets["validation"]' has the proper format
# validation_reasons = [example['reason'] for example in tokenized_datasets["validation"]]
from tqdm import tqdm
generated_reasons = []
data_files = {
    "train": file_name,
    "validation": val_name
}
dataset = load_dataset("json", data_files=data_files)
validation_reasons = []
for example in tqdm(dataset["validation"]):
    query = example['query']
    documents = example['unsorted_docs']
    # print("query", query)
    generated_reason = generate_reason(query, documents)
    validation_reasons.append(example['reason'])
    print("generated_reason", generated_reason)
    generated_reasons.append(generated_reason)

# Compute ROUGE scores
rouge_scores = compute_rouge_scores(generated_reasons, validation_reasons)
print("Validation ROUGE Scores:", rouge_scores)
align_scores = compute_align_scores(generated_reasons, validation_reasons)
print("Validation Align Scores:", align_scores)

  1%|▋                                                              | 1/100 [00:02<04:11,  2.54s/it]

generated_reason [1] > [2] > [3] > [4] Reasoning: - Passage [1] directly addresses the query about how contaminated our children are, focusing on serving styles of snack vegetables. - Passage [2] discusses the impact of dietary contaminants on children, which is relevant to the query about how contaminated our children are. - Passage [3] discusses the effects of dietary contaminants on children, which is relevant to the query about how contaminated our children are. - Passage [4] discusses the effects of dietary contaminants on children, which is relevant to the query about how contaminated our children are.


  2%|█▎                                                             | 2/100 [00:04<03:47,  2.32s/it]

generated_reason [1] > [2] > [3] > [4] Reasoning: - Passage [1] directly discusses the association between dietary nutrients and endometrial cancer risk, which is relevant to the query about cancer and the animal-to-plant protein ratio. - Passage [2] discusses the association between dietary protein and endometrial cancer risk, which is relevant to the query about cancer and the animal-to-plant protein ratio. - Passage [3] discusses the association between dietary protein and endometrial cancer risk, which is relevant to the query about cancer and the animal-to-plant protein ratio. - Passage [4]


  3%|█▉                                                             | 3/100 [00:06<03:37,  2.24s/it]

generated_reason [1] > [2] > [3] > [4] Reasoning: - Passage [1] directly discusses the health benefits of plant-based diets, which is relevant to the query about how plant-based diets may extend our lives. - Passage [2] discusses the role of Nrf2 in the pro-ageing signaling, which is relevant to the query about how plant-based diets may extend our lives. - Passage [3] discusses the role of dietary antioxidants in the nonequilibrium systems that control redox balance, cell signaling, and cell function, which is relevant to the query about how plant-based diets


  3%|█▉                                                             | 3/100 [00:08<04:36,  2.85s/it]


KeyboardInterrupt: 

In [13]:
# model.device

device(type='cpu')

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
# Load the trained model and tokenizer
model_path = './t5_results/checkpoint-500'  # Adjust path if your trained model is saved elsewhere
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [1]:
from alignscore import AlignScore
import json
pred_path = "/home/yuj49/DIAYN/data/T5_results/GPT3.5_generated_reasons.txt"
refer_path = "/home/yuj49/DIAYN/data/GPT3.5/msmarco_test.jsonl"


references = []
predictions = []

with open(pred_path, "r") as f1, open(refer_path, "r") as f2:
    for line in f1.readlines():
        predictions.append(line)
    for line in f2.readlines():
        data = json.loads(line)
        references.append(data["reason"])
        
scorer = AlignScore(model='roberta-base', batch_size=32, device='cuda:0', ckpt_path='/home/yuj49/BioLaySumm2024-evaluation_scripts/models/AlignScore/AlignScore-base.ckpt', evaluation_mode='nli_sp')
# score = scorer.score(contexts=["[2] > [4] > [5] > [1] > [3] I ranked passage [2] as the most relevant because it directly addresses the exposure of children to multiple dietary contaminants, highlighting the vulnerability of children to toxic exposure. Passage [4] follows as it discusses the association between dietary history and plasma clastogenic factors in children from regions contaminated by the Chernobyl accident, providing insights into the impact of contamination on children's health. Passage [5] is ranked next as it focuses on the neurodevelopmental impacts of methyl mercury toxicity on children, emphasizing the economic costs and consequences of exposure. Passage [1] is ranked fourth as it primarily discusses children's preferences for snack vegetables and subscription services, which is less directly related to the query about contamination. Passage [3] is ranked last as it discusses awareness of mercury in fish among consumers, which is indirectly related to the query about contamination of children."],
#                      claims=['[1] > [2] > [3] > [4] Reasoning: - Passage [1] directly addresses the query about how contaminated our children are, focusing on serving styles of snack vegetables. - Passage [2] discusses the impact of dietary contaminants on children, which is relevant to the query about how contaminated our children are. - Passage [3] discusses the effects of dietary contaminants on children, which is relevant to the query about how contaminated our children are. - Passage [4] discusses the effects of dietary contaminants on children, which is relevant to the query about how contaminated our children are.'])
scores = scorer.score(contexts=references,claims=predictions)
averg = sum(scores)/len(scores)
averg

/home/yuj49/anaconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-11 22:36:30.306830: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-11 22:36:30.352505: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 22:36:30.993823: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning:

0.5209902613532058